In [ ]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_community.tools import YouTubeSearchTool


 Arxiv, Wikipedia & YouTube Search Tools
 This section initializes research-related tools:
 - ArxivQueryRun for academic paper search
 - WikipediaQueryRun for wiki-based information retrieval
 - YouTubeSearchTool for fetching relevant YouTube video results
 These tools will be used by the agent to gather external knowledge.

In [4]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os


### LLM Initialization (Groq)
- Initialize the Groq Llama 3.1 model using the API key from environment variables.

In [5]:
groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E3B13320D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E3B11078D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [7]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


defines a YouTube search tool for LangChain using the @tool decorator. It takes a search query, retrieves up to 10 YouTube results using YoutubeSearch, extracts details such as title, channel, duration, and full video URL, and returns them in a structured dictionary format for use by an agent.

In [84]:
from youtube_search import YoutubeSearch
from langchain.tools import tool

@tool
def youtube_search(query: str) -> dict:
    """
    Search YouTube videos for the given query.
    """
    raw = YoutubeSearch(query, max_results=10).to_dict()  # returns list

    videos = []
    for v in raw:
        videos.append({
            "title": v.get("title"),
            "channel": v.get("channel"),
            "duration": v.get("duration"),
            "url": f"https://www.youtube.com{v.get('url_suffix')}"
        })

    return {"videos": videos}


In [8]:
from langchain_classic.agents import create_tool_calling_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

These imports load the necessary components for building a tool-calling agent in LangChain. create_tool_calling_agent creates agents that can automatically decide when to use tools, @tool is used to define custom tools, and ChatPromptTemplate with MessagesPlaceholder helps construct structured chat prompts for the agent.

In [9]:
# Tool
@tool
def multiply(a: int, b: int):
    """Multiply two numbers."""
    return a * b

Configures a tool-enabled AI agent using a custom prompt and a set of tools (multiply, Wikipedia, and Arxiv). The prompt defines how the agent should behave, while the create_tool_calling_agent function connects the LLM with the available tools. The resulting agent can intelligently decide when to call these tools to answer user queries.

In [10]:

tools = [multiply,wiki,arxiv]

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools ONLY when necessary."),
    MessagesPlaceholder("messages"),
    MessagesPlaceholder("agent_scratchpad") 
])

# Agent
agent = create_tool_calling_agent(
    llm,
    tools=tools,
    prompt=prompt
)


Creates an AgentExecutor, which is responsible for running the agent with the specified tools. It manages the full execution loop—handling user inputs, deciding when to call tools, collecting intermediate steps, and producing the final response. Enabling verbose=True prints detailed execution traces for easier debugging and understanding of the agent’s reasoning.

In [11]:
from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langch

This code sends a user query—“Tell me about Langsmith”—to the agent via agent_executor.invoke(). The agent processes the message, optionally uses tools, and returns the final response. The result is stored in result1 and then displayed.

In [12]:
result1 = agent_executor.invoke({
    "messages": [
        {"role": "user", "content": "Tell me about Langsmith"}
    ]
})
result1



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag
Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag
Invoking: `wikipedia` with `{'query': 'Langsmith'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag
Invoking: `wikipedia` with `{'query': 'Langs

{'messages': [{'role': 'user', 'content': 'Tell me about Langsmith'}],
 'output': "It seems that there is not enough information available about 'Langsmith' in Wikipedia. If you'd like to know more about a specific topic, I'd be happy to try and help you find more information."}

In [15]:
result1['output']

"It seems that there is not enough information available about 'Langsmith' in Wikipedia. If you'd like to know more about a specific topic, I'd be happy to try and help you find more information."

In [16]:
#agent_executor.invoke({"input":"Tell me about Langsmith"})
result2 = agent_executor.invoke({
    "messages": [
        {"role": "user", "content": "multiply 2 by 100"}
    ]
})
result2['output']



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 100}`


200The result of the multiplication is 200.

> Finished chain.


'The result of the multiplication is 200.'

In [17]:

result3 = agent_executor.invoke({
    "messages": [
        {"role": "user", "content": "Provide brife discription about the research paper 'Attention Is All You Need'"}
    ]
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention Is All You Need research paper'}`


Published: 2006-01-22
Title: This paper has been withdrawn
Authors: This paper has been withdrawn
Summary: This paper has been withdrawn
Invoking: `arxiv` with `{'query': 'Attention Is All You Need Vaswani et al 2017'}`
responded: It seems that the paper 'Attention Is All You Need' is not found in the arXiv database. This is likely because the paper was actually published in 2017, not 2006. Let me try again with the correct year.



Published: 2015-06-15
Title: On the sensitivity of extracting the astrophysical cross section factor of the 12C(a,g) reaction from existing data [Comment on Schuermann et al. Phys. Lett. B711(2012)35]
Authors: Moshe Gai
Summary: We address a conflict
Invoking: `arxiv` with `{'query': 'Attention Is All You Need Vaswani et al 2017'}`


Published: 2015-06-15
Title: On the sensitivity of extracting the astrophysical cross section factor of t

In [18]:
result3['output']

'The paper "Attention Is All You Need" was published in 2017 by Vaswani et al. It introduced a new neural network architecture called the transformer that is based entirely on self-attention mechanisms, eliminating the need for recurrent neural networks (RNNs) and convolutional neural networks (CNNs) in sequence-to-sequence models. This architecture has since become a widely used building block in many natural language processing (NLP) and other applications.'

These imports prepare the components needed to build a custom Retrieval-Augmented Generation (RAG) tool. The loader fetches web content, the text splitter breaks it into chunks, OpenAI embeddings convert text into vector representations, and FAISS stores and retrieves those vectors efficiently for semantic search.

In [19]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


code initializes a Hugging Face embedding model (all-MiniLM-L6-v2) using HuggingFaceEmbeddings. The model converts text into numerical vector embeddings, which are useful for tasks like semantic search, similarity comparison, and building RAG pipelines.

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

f:\AgenticAI\Workplace\tf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


code loads content from the LangSmith documentation website using WebBaseLoader, splits the text into manageable chunks with RecursiveCharacterTextSplitter, and converts the chunks into vector embeddings using a Hugging Face model. These embeddings are stored in a FAISS vector database, which is then converted into a retriever for semantic search and RAG-based question answering.

In [29]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E3B36149D0>, search_kwargs={})

In [30]:
from langchain_classic.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

This code wraps the FAISS-based retriever into a LangChain tool using create_retriever_tool. The resulting tool, named "langsmith-search", allows the agent to semantically search the LangSmith documentation whenever needed.

In [31]:
retriever_tool

Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E3B1BAF4C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E3B36149D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001E3B1BAF600>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E3B36149D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), d

In [85]:

tools = [multiply,  arxiv, retriever_tool,wiki, youtube_search]

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. When a tool is used, include the tool results in the final answer."),
    MessagesPlaceholder("messages"),
    MessagesPlaceholder("agent_scratchpad") 
])

# Agent
agent = create_tool_calling_agent(
    llm,
    tools=tools,
    prompt=prompt
)


In [86]:

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langch

In [ ]:
result1_updated = agent_executor.invoke({
    "messages": [
        {"role": "user", "content": "Tell me about Langsmith"}
    ]
})
result1_updated



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


LangSmith meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. For more information, see the Trust Center.
​Workflow
LangSmith combines observability, evaluation, deployment, and platform setup in one integrated workflow—from local development to production.

LangSmith docs - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS

{'messages': [{'role': 'user', 'content': 'Tell me about Langsmith'}],
 'output': "The provided information about Langsmith is a documentation and guide for the LangSmith platform, which is a tool for developing, debugging, and deploying large language model (LLM) applications. It provides features such as observability, evaluation, deployment, and platform setup, and is framework-agnostic, allowing users to use it with or without LangChain's open-source libraries."}

In [87]:
result4 = agent_executor.invoke({
    "messages": [
        {"role": "user", "content": "Provide Langsmith videos from youtube along with respective links"}
    ]
})
result4



> Entering new AgentExecutor chain...

Invoking: `youtube_search` with `{'query': 'Langsmith videos'}`


{'videos': [{'title': 'What Is LangSmith? Explained in 5 Minutes', 'channel': 'LangChain', 'duration': '5:25', 'url': 'https://www.youtube.com/watch?v=kYtnLaJeia8&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D'}, {'title': 'LangChain vs LangGraph vs LangSmith', 'channel': 'codebasics', 'duration': '10:16', 'url': 'https://www.youtube.com/watch?v=vJOGC8QJZJQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D'}, {'title': 'LangSmith in 10 Minutes', 'channel': 'LangChain', 'duration': '9:21', 'url': 'https://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvc9IHCQkLCgGHKiGM7w%3D%3D'}, {'title': 'LangSmith Crash Course | LangSmith Tutorial for Beginners | Observability in GenAI | CampusX', 'channel': 'CampusX', 'duration': '2:07:40', 'url': 'https://www.youtube.com/watch?v=4FFspU4riHk&t=113s&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D'}, {'title': 'LangGraph vs LangChain vs LangFlow vs LangSmith : Which O

{'messages': [{'role': 'user',
   'content': 'Provide Langsmith videos from youtube along with respective links'}],
 'output': 'The Langsmith videos from YouTube along with their respective links are as follows:\n\n1. What Is LangSmith? Explained in 5 Minutes - https://www.youtube.com/watch?v=kYtnLaJeia8&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n2. LangChain vs LangGraph vs LangSmith - https://www.youtube.com/watch?v=vJOGC8QJZJQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n3. LangSmith in 10 Minutes - https://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvc9IHCQkLCgGHKiGM7w%3D%3D\n4. LangSmith Crash Course | LangSmith Tutorial for Beginners | Observability in GenAI | CampusX - https://www.youtube.com/watch?v=4FFspU4riHk&t=113s&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n5. LangGraph vs LangChain vs LangFlow vs LangSmith : Which One To Use & Why? - https://www.youtube.com/watch?v=ldBsvhjEREc&t=431s&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n6. Langsmith LLMOPS Platform By Langchain-Debug ,Monitor

In [88]:
result4['output']

'The Langsmith videos from YouTube along with their respective links are as follows:\n\n1. What Is LangSmith? Explained in 5 Minutes - https://www.youtube.com/watch?v=kYtnLaJeia8&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n2. LangChain vs LangGraph vs LangSmith - https://www.youtube.com/watch?v=vJOGC8QJZJQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n3. LangSmith in 10 Minutes - https://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvc9IHCQkLCgGHKiGM7w%3D%3D\n4. LangSmith Crash Course | LangSmith Tutorial for Beginners | Observability in GenAI | CampusX - https://www.youtube.com/watch?v=4FFspU4riHk&t=113s&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n5. LangGraph vs LangChain vs LangFlow vs LangSmith : Which One To Use & Why? - https://www.youtube.com/watch?v=ldBsvhjEREc&t=431s&pp=ygUQTGFuZ3NtaXRoIHZpZGVvcw%3D%3D\n6. Langsmith LLMOPS Platform By Langchain-Debug ,Monitor And Build Production Grade LLM Application - https://www.youtube.com/watch?v=3Gcm27l-uyQ&pp=ygUQTGFuZ3NtaXRoIHZpZGVvc9IHCQkLCg

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.13.2
aiosignal                1.4.0
annotated-types          0.7.0
anyio                    4.11.0
arxiv                    2.3.1
asttokens                2.4.1
attrs                    25.4.0
beautifulsoup4           4.14.2
bs4                      0.0.2
certifi                  2025.11.12
charset-normalizer       3.4.4
colorama                 0.4.6
comm                     0.2.1
dataclasses-json         0.6.7
debugpy                  1.8.1
decorator                5.1.1
distro                   1.9.0
executing                2.0.1
faiss-cpu                1.13.0
feedparser               6.0.12
filelock                 3.20.0
frozenlist               1.8.0
fsspec                   2025.10.0
greenlet                 3.2.4
groq                     0.36.0
h11                      0.16.0
httpcore                 1.0.9
httpx                    0.28.1
httpx-sse                0.4.3
huggingface-hub          0.36.0
idna                     3.11
importlib_metadata       8.7.0
ipykernel                7.1.0
ipython                  9.7.0
ipython_pygments_lexers  1.1.1
jedi                     0.19.2
Jinja2                   3.1.6
jiter                    0.12.0
joblib                   1.5.2
jsonpatch                1.33
jsonpointer              3.0.0
jupyter_client           8.6.0
jupyter_core             5.7.1
langchain                1.0.8
langchain-classic        1.0.0
langchain-community      0.4.1
langchain-core           1.1.0
langchain-groq           1.0.1
langchain-huggingface    1.0.1
langchain-openai         1.0.3
langchain-text-splitters 1.0.0
langgraph                1.0.3
langgraph-checkpoint     3.0.1
langgraph-prebuilt       1.0.5
langgraph-sdk            0.2.9
langsmith                0.4.46
MarkupSafe               3.0.3
marshmallow              3.26.1
matplotlib-inline        0.1.6
mpmath                   1.3.0
multidict                6.7.0
mypy_extensions          1.1.0
nest-asyncio             1.6.0
networkx                 3.6
numpy                    2.3.5
openai                   2.8.1
orjson                   3.11.4
ormsgpack                1.12.0
packaging                23.2
parso                    0.8.3
pillow                   12.0.0
pip                      25.3
platformdirs             4.5.0
prompt-toolkit           3.0.43
propcache                0.4.1
psutil                   5.9.8
pure-eval                0.2.2
pydantic                 2.12.4
pydantic_core            2.41.5
pydantic-settings        2.12.0
Pygments                 2.17.2
python-dateutil          2.9.0.post0
python-dotenv            1.2.1
pywin32                  306
PyYAML                   6.0.3
pyzmq                    25.1.2
regex                    2025.11.3
requests                 2.32.5
requests-toolbelt        1.0.0
safetensors              0.7.0
scikit-learn             1.7.2
scipy                    1.16.3
sentence-transformers    5.1.2
setuptools               80.9.0
sgmllib3k                1.0.0
six                      1.16.0
sniffio                  1.3.1
soupsieve                2.8
SQLAlchemy               2.0.44
stack-data               0.6.3
sympy                    1.14.0
tenacity                 9.1.2
threadpoolctl            3.6.0
tiktoken                 0.12.0
tokenizers               0.22.1
torch                    2.9.1
tornado                  6.4
tqdm                     4.67.1
traitlets                5.14.3
transformers             4.57.1
typing_extensions        4.15.0
typing-inspect           0.9.0
typing-inspection        0.4.2
urllib3                  2.5.0
wcwidth                  0.2.13
wheel                    0.45.1
wikipedia                1.4.0
xxhash                   3.6.0
yarl                     1.22.0
youtube-search           2.1.2
zipp                     3.23.0
zstandard                0.25.0